# DACON "AI야, 진짜 뉴스를 찾아줘!" - 북극506번지  
  
주 언어 : Pytorch
  

In [ ]:
# Hugging Face의 Transformers
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv('/content/gdrive/My Drive/NH공모전/data/news_train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('/content/gdrive/My Drive/NH공모전/data/news_test.csv', encoding = 'utf-8-sig')

## 시간 측정 시작

In [ ]:
import time
start_time = time.time()

## Library 불러오기

In [ ]:
# 데이터 전처리를 위한 라이브러리
import numpy as np
import pandas as pd
import warnings
import random
import time
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_row', 500)

# 모델링을 위한 라이브러리
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import torch
import transformers

## Tokenizer, pretraind_embedding, Model 불러오기  
  
> Bert Tokenizer 사용 `bert-base-multilingual-cased`  
> Bert pretrained model 사용 `bert-base-multilingual-cased`  

In [ ]:
# 디바이스 설정
device = torch.device("cuda")

# 모델
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)
model.load_state_dict(torch.load('./gdrive/My Drive/NH공모전/제출/룰추가한모델42/NH_final_model.pt'))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

<All keys matched successfully>

## 형태소 분석 + 전처리

In [ ]:
# === 파생변수 생성 === 

max_ord = test.groupby('n_id').max(ord).ord
test = pd.merge(test, max_ord, on='n_id')
test['ordp'] = test.ord_x/test.ord_y
test['ordp_s'] = (test.ordp*100).astype(int).astype(str) # 최종적으로 사용할 변수

In [ ]:
# ==============================================
#       Rule-based prediction을 위한 전처리
# ==============================================

# === content가 동일하면 같은 info 값을 가지는 경우 ===

traind = train[['content', 'info']].drop_duplicates()
traind = traind.groupby(['content']).count()
not_unique_list = traind.query('info == 2').index.to_list() # info 값이 unique하지 않은 학습데이터의 content list

# 가짜뉴스
fake_content = train.query('info == 1 and content not in @not_unique_list').content
fake_content = list(fake_content.unique())
absolute_fake = test.query('content in @fake_content')
absolute_fakeid = list(absolute_fake.id)

# 진짜뉴스
real_content = train.query('info == 0 and content not in @not_unique_list').content
real_content = list(real_content.unique())
absolute_real = test.query('content in @real_content')
absolute_realid = list(absolute_real.id)

# === content가 동일한데 다른 info 값을 가지는 경우 ===

# content = "데이터에서 찾은 AI 속보"
ai = test.query('content == "데이터에서 찾은 AI 속보"')
ai_id = list(ai.id)
info = ai.apply(lambda x: 0 if x['ordp'] < 0.5 else 1, axis=1)

absolute_id = absolute_fakeid + absolute_realid + ai_id

# === 기사 길이가 1인 경우 ===
# length_one = test[test['ord_y']==1]
# # length_oneid = list(length_one.id)
# # length_oneid = set(length_oneid) - set(absolute_id) # [id for id in length_oneid if id not in absolute_id] # 중복 체크
# length_oneid = list(set(length_one.id) - set(absolute_id))
# absolute_id = absolute_id + length_oneid

# === 추가적인 개별 check ===
test['temp'] = test['content'].str.slice(start=0, stop=9)
extra = test.query('content == "참여 방법:" or temp == "무료종목상담 일정" or temp == "http://ww"')
# extra_id = list(extra.id)
# extra_id = set(extra_id) - set(absolute_id) # [id for id in extra_id if id not in absolute_id] # 중복 체크
extra_id = list(set(extra.id) - set(absolute_id))
absolute_id = absolute_id + extra_id

In [ ]:
# ===========================================
#       BERT prediction을 위한 전처리
# ===========================================

test2 = test.query('id not in @absolute_id')
sentences = test2['content'] + ' ' + test2['ordp_s'] + ' ' + test2['title']

sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

MAX_LEN = 100

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)

batch_size = 32

test_data = torch.utils.data.TensorDataset(test_inputs, test_masks)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


## 예측

> Step1. Rule-based prediction  
  * train data를 기반으로 한 rule-based prediction 
  * train data의 `content`와 test data의 `content`가 동일한 경우, train data의 `info`로 test data의 `info` 결정   
  * train data 중, `content`가 `데이터에서 찾은 AI 속보`임에도 제목, 주변 뉴스의 내용, 뉴스의 위치에 따라 `info`가 다른 것을 확인할 수 있었음. 이와 같은 데이터는 `ord_p`가 0.5 미만일 경우 `info`가 0, 50% 이상일 경우 `info`가 1이었으므로 이를 Rule에 반영함.  

> Step2. BERT prediction  
  * Rule-based prediction이 불가능한, 즉 train data에 없는 test data에 대해서는 BERT모델을 활용하여 예측

> Step3. Final prediction  
  * Step1과 Step2의 결과를 합친 최종 예측

In [ ]:
# === 시간 표시 함수 ===
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# ========================================
#       Step1. Rule-based prediction
# ========================================

traind = train[['content', 'info']].drop_duplicates()
traind = traind.groupby(['content']).count()
not_unique_list = traind.query('info == 2').index.to_list() # info 값이 unique하지 않은 학습데이터의 content list

# === content가 동일하면 같은 info 값을 가지는 경우 ===

# 가짜 뉴스 판별
pred_fake = {'id':absolute_fakeid, 'info':1}

# 진짜 뉴스 판별 
pred_real = {'id':absolute_realid, 'info':0}

# === content가 동일한데 다른 info 값을 가지는 경우 ===
pred_ai = {'id':ai_id, 'info':info}

# === 기사 길이가 1인 경우 ===
# pred_lenone = {'id':length_oneid, 'info':0}

# === 추가적인 개별 check ===
pred_extra = {'id':extra_id, 'info':1}


# ========================================
#       Step2. BERT prediction
# ========================================

# 평가모드로 변경
model.eval()

# 시작 시간 설정
t0 = time.time()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
preds = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):

    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 예측 
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    pred_flat = np.argmax(logits, axis=1).flatten().tolist()
    preds.append(pred_flat)

  Batch   100  of  2,758.    Elapsed: 0:00:10.
  Batch   200  of  2,758.    Elapsed: 0:00:20.
  Batch   300  of  2,758.    Elapsed: 0:00:29.
  Batch   400  of  2,758.    Elapsed: 0:00:39.
  Batch   500  of  2,758.    Elapsed: 0:00:49.
  Batch   600  of  2,758.    Elapsed: 0:00:59.
  Batch   700  of  2,758.    Elapsed: 0:01:09.
  Batch   800  of  2,758.    Elapsed: 0:01:18.
  Batch   900  of  2,758.    Elapsed: 0:01:28.
  Batch 1,000  of  2,758.    Elapsed: 0:01:38.
  Batch 1,100  of  2,758.    Elapsed: 0:01:48.
  Batch 1,200  of  2,758.    Elapsed: 0:01:58.
  Batch 1,300  of  2,758.    Elapsed: 0:02:07.
  Batch 1,400  of  2,758.    Elapsed: 0:02:17.
  Batch 1,500  of  2,758.    Elapsed: 0:02:27.
  Batch 1,600  of  2,758.    Elapsed: 0:02:37.
  Batch 1,700  of  2,758.    Elapsed: 0:02:47.
  Batch 1,800  of  2,758.    Elapsed: 0:02:56.
  Batch 1,900  of  2,758.    Elapsed: 0:03:06.
  Batch 2,000  of  2,758.    Elapsed: 0:03:16.
  Batch 2,100  of  2,758.    Elapsed: 0:03:26.
  Batch 2,200

In [ ]:
# ========================================
#       Step3. Final Prediction
# ========================================

preds = sum(preds, [])
submission_ids = list(test2['id'])
pred_bert = {"id":submission_ids, "info":preds}
sub = pd.concat([pd.DataFrame(pred_fake), pd.DataFrame(pred_real), pd.DataFrame(pred_ai), pd.DataFrame(pred_extra), pd.DataFrame(pred_bert)], axis=0) # , pd.DataFrame(pred_lenone)
end_time = time.time()

sub = sub.set_index('id')
sub = sub.reindex(list(test.id))
sub = sub.reset_index()

print('Test time : {}'.format(end_time-start_time)) 

Test time : 307.0316689014435


## 제출

In [ ]:
sub.to_csv('./gdrive/My Drive/NH공모전/제출/곧지움_NH_final_test_1.csv', index=False)